# Train Noise2Void
Project cloned from: https://github.com/juglab/n2v

$ git clone https://github.com/juglab/n2v.git

Change into its directory and install it:

$ pip install -e .

During operation it can be useful to evaluate GPU usage.  To do so, open a terminal and execute the following command:
watch -n 0.5 nvidia-smi

In [2]:
import os
from n2v.models import N2VConfig, N2V
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
import numpy as np
from tifffile import imread
from csbdeep.io import save_tiff_imagej_compatible

/home2/kdean/.conda/envs/n2venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home2/kdean/.conda/envs/n2venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home2/kdean/.conda/envs/n2venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home2/kdean/.conda/envs/n2venv/lib/python3.6/site-pack

# Training Data Preparation

* Create directory for saving data.
* Create DataGenerator-object - helps load data and extract patches for training and validation.
* Load all .tif files - Important to know how the data is loaded, and you can check this by printing the shape.  In some cases, OME-XML for some reason loads all of the volumes from the data.
* Display shape of the images



In [3]:
# Specify where the data is located
data_directory = '/project/shared/dean_wbmf/denoise_attempt'
image_name = 'ch02_subset.tif'
test = imread(os.path.join(data_directory,image_name))
print(test.shape)

# Create a folder for the results
if not os.path.isdir(os.path.join(data_directory,'denoised')):
    os.mkdir(os.path.join(data_directory,'denoised'))

# Create Data Generator
datagen = N2V_DataGenerator()

(150, 920, 1172)


If the data does not trigger the OME-XML import, you can load each image from the directory as follows...

In [ ]:
imgs = datagen.load_imgs_from_directory(directory=data_directory, dims='TZYX')
print(imgs[0].shape)

If the data does trigger OME-XML import, you can load only the first image from the directory, which will then automatically load the rest too.

In [ ]:
files = [os.path.join(data_directory,image_name)]
imgs = datagen.load_imgs(files, dims='TZYX')
print(imgs[0].shape)

In [4]:
files = [os.path.join(data_directory,image_name)]
imgs = datagen.load_imgs(files, dims='ZYX')
print(imgs[0].shape)

(1, 150, 920, 1172, 1)


# Generate Training Patches
* default = 32,64,64
* training images must be evenly divisible by 4 along axes XYZ 

In [ ]:
patches = datagen.generate_patches_from_list(imgs[:1], shape=(32, 64, 64))

# Specify Training Settings

In [41]:
# Number of Epochs
numberEpochs = 40

# default = :600
X = patches[:600]
X_val = patches[600:]
config = N2VConfig(X, unet_kern_size=3, train_steps_per_epoch=int(X.shape[0]/128),
                   train_epochs=numberEpochs,train_loss='mse', batch_norm=True, train_batch_size=4,
                   n2v_perc_pix=0.198, n2v_patch_shape=(32, 64, 64), n2v_manipulator='uniform_withCP',
                   n2v_neighborhood_radius=5)
vars(config)
model_name = str(numberEpochs) + 'epochs'
model = N2V(config=config, name=model_name, basedir=data_directory)

# Run N2V Training

In [ ]:
history = model.train(X, X_val)
print(sorted(list(history.history.keys())))
model.export_TF()

259 blind-spots will be generated per training patch of size (32, 64, 64).


Preparing validation data: 100%|██████████| 7464/7464 [00:04<00:00, 1637.13it/s]


Epoch 1/40
4/4 [==============================] - 1225s 306s/step - loss: 1.0262 - n2v_mse: 1.0262 - n2v_abs: 0.8215 - val_loss: 1.1113 - val_n2v_mse: 1.1113 - val_n2v_abs: 0.7742
Epoch 2/40
4/4 [==============================] - 1223s 306s/step - loss: 0.9357 - n2v_mse: 0.9357 - n2v_abs: 0.7909 - val_loss: 1.2679 - val_n2v_mse: 1.2679 - val_n2v_abs: 0.7890
Epoch 3/40
4/4 [==============================] - 1224s 306s/step - loss: 0.8551 - n2v_mse: 0.8551 - n2v_abs: 0.7399 - val_loss: 2.5527 - val_n2v_mse: 2.5527 - val_n2v_abs: 1.1459
Epoch 4/40
4/4 [==============================] - 1224s 306s/step - loss: 0.4578 - n2v_mse: 0.4578 - n2v_abs: 0.5513 - val_loss: 2.4181 - val_n2v_mse: 2.4181 - val_n2v_abs: 1.0463
Epoch 5/40
4/4 [==============================] - 1225s 306s/step - loss: 0.4953 - n2v_mse: 0.4953 - n2v_abs: 0.5470 - val_loss: 2.3130 - val_n2v_mse: 2.3130 - val_n2v_abs: 1.0403
Epoch 6/40
4/4 [==============================] - 1224s 306s/step - loss: 0.3821 - n2v_mse: 0.3821 -

# Test the Training and Export a Denoised Image

In [9]:
files = [os.path.join(data_directory,image_name)]
imgs = datagen.load_imgs(files, dims='ZYX')
print(imgs[0].shape)

model_name = str(40) + 'epochs'
model = N2V(config=None, name=model_name, basedir=os.path.join(data_directory))

# Load the image, and predict the denoised image.
pred = model.predict(imgs, axes='ZYX', n_tiles=(2,4,4))
save_tiff_imagej_compatible(os.path.join(data_directory, 'denoised','denoised.tif'), pred, 'ZYX')

(1, 150, 920, 1172, 1)
Loading network weights from 'weights_best.h5'.


AttributeError: 'list' object has no attribute 'shape'